In [ ]:
# Resume Redacting

In [ ]:
import pdfrw
import PyPDF2 
from pdfrw import PdfReader, PdfWriter

In [ ]:
path = '/home/subbu/PRML/Internship/p2/out/'
file = 'out.pdf'

In [ ]:
pdf = PdfReader(path+file)

In [ ]:
#print(len(pdf.pages))
pdf.pages[0]

In [ ]:
pdfFileObj = open(path+file, 'rb')

In [ ]:
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

In [ ]:
print(pdfReader.getPage(0).extractText())

In [ ]:
# converting pdf into jpeg

In [10]:
from pdf2image import convert_from_path

In [22]:
pages = convert_from_path('in/Resume3.pdf', dpi=800)
i = 0
for page in pages:
    page.save('out/imj'+str(i)+'.jpg', 'JPEG')
    i += 1

In [58]:
# using google api

In [75]:
import google.cloud.dlp
import mimetypes
import sys
import os
import subprocess
from gcloud import storage
from fpdf import FPDF
from PIL import Image

In [68]:
filename = 'out/imj0.jpg'
mime_type=None
project = 'rankingtranscript'
output_filename = 'out/imj-modf.png'
doc = '/home/subbu/PRML/Internship/p2/in/Resume4.docx'

In [50]:
os.system("libreoffice --headless -convert-to pdf --outdir /home/subbu/PRML/Internship/p2/in/ "+doc)

0

In [53]:
# following 3 cells are download from cloud

storage_client = storage.Client.from_service_account_json(
        '/home/subbu/PRML/Internship/p1/rankingtranscript-96316cee08ba.json')
bucket = storage_client.get_bucket('rankingtranscript1')

In [54]:
blob = bucket.get_blob('Resume3.pdf')

In [55]:
blob.download_to_filename('in/ResumeD.pdf')

In [37]:



dlp = google.cloud.dlp_v2.DlpServiceClient()

if mime_type is None:
    mime_guess = mimetypes.MimeTypes().guess_type(filename)
    mime_type = mime_guess[0] or "application/octet-stream"

supported_content_types = {
    None: 0,
    "image/jpeg": 1,
    "image/bmp": 2,
    "image/png": 3,
    "image/svg": 4,
    "text/plain": 5,
}
content_type_index = supported_content_types.get(mime_type, 0)


with open(filename, mode="rb") as f:
    byte_item = {"type": content_type_index, "data": f.read()}

custom_regexes = ["[0-9]{10}" # 10 digit number
                 
                 ] # add regular expressions here

regexes = [
        {
            "info_type": {"name": "CUSTOM_REGEX_{}".format(i)},
            "regex": {"pattern": custom_regex},
        }
        for i, custom_regex in enumerate(custom_regexes)
    ]
custom_info_types = regexes

info_types = [{"name": "EMAIL_ADDRESS"}]

inspect_config = {
        "info_types": info_types,
        "custom_info_types": custom_info_types,
    }

image_redaction_configs = []

for info_type in info_types:
    image_redaction_configs.append({"info_type": info_type})
image_redaction_configs.append({"info_type": {"name":"CUSTOM_REGEX_0"}})


parent = dlp.project_path(project)


In [24]:
# calling google api

response = dlp.redact_image(
    parent,
    inspect_config=inspect_config,
    image_redaction_configs=image_redaction_configs,
    byte_item=byte_item,
)

In [69]:
# getting jpg

with open(output_filename, mode="wb") as f:
    f.write(response.redacted_image)

In [77]:
# conversion to pdf

cover = Image.open("out/imj-modf" + ".jpg")
width, height = cover.size

pdf = FPDF(unit = "pt", format = [width, height])

for image in ['out/imj-modf.jpg']:
    pdf.add_page()
    pdf.image(image,0,0)
pdf.output("out/Resume.pdf", "F")

''

In [ ]:
# upload to cloud
blob2 = bucket.blob(file_name)
blob.upload_from_filename('out/')

In [17]:
# data of what all are present

with open(filename, mode="rb") as f:
        item = {"byte_item": {"type": content_type_index, "data": f.read()}}
        
response = dlp.inspect_content(
    parent,
    inspect_config=inspect_config,
    item=item,
)


if response.result.findings:
    for finding in response.result.findings:
        try:
            print("Quote: {}".format(finding.quote))
        except AttributeError:
            pass
        print("Info type: {}".format(finding.info_type.name))
        print("Likelihood: {}".format(finding.likelihood))
else:
    print("No findings.")

Quote: 
Info type: EMAIL_ADDRESS
Likelihood: 4
Quote: 
Info type: CUSTOM_REGEX_0
Likelihood: 5
Quote: 
Info type: CUSTOM_REGEX_0
Likelihood: 5


In [78]:
content_type_index

1